In [3]:
import pandas as pd

In [4]:
df = pd.read_csv("spotify_millsongdata.csv")

In [5]:
df.head(5)

,artist,song,link,text
0,ABBA,Ahe's My Kind Of Girl,/a/abba/ahes+my+kind+of+girl_20598417.html,"Look at her face, it's a wonderful face \r\nA..."
1,ABBA,"Andante, Andante",/a/abba/andante+andante_20002708.html,"Take it easy with me, please \r\nTouch me gen..."
2,ABBA,As Good As New,/a/abba/as+good+as+new_20003033.html,I'll never know why I had to go \r\nWhy I had...
3,ABBA,Bang,/a/abba/bang_20598415.html,Making somebody happy is a question of give an...
4,ABBA,Bang-A-Boomerang,/a/abba/bang+a+boomerang_20002668.html,Making somebody happy is a question of give an...


In [6]:
df.tail(5)

,artist,song,link,text
57645,Ziggy Marley,Good Old Days,/z/ziggy+marley/good+old+days_10198588.html,Irie days come on play \r\nLet the angels fly...
57646,Ziggy Marley,Hand To Mouth,/z/ziggy+marley/hand+to+mouth_20531167.html,Power to the workers \r\nMore power \r\nPowe...
57647,Zwan,Come With Me,/z/zwan/come+with+me_20148981.html,all you need \r\nis something i'll believe \...
57648,Zwan,Desire,/z/zwan/desire_20148986.html,northern star \r\nam i frightened \r\nwhere ...
57649,Zwan,Heartsong,/z/zwan/heartsong_20148991.html,come in \r\nmake yourself at home \r\ni'm a ...


In [7]:
df.shape

(57650, 4)

In [8]:
df.isnull().sum()

artist    0
song      0
link      0
text      0
dtype: int64

In [9]:
df =df.sample(5000).drop('link', axis=1).reset_index(drop=True)

In [10]:
df.head(10)

,artist,song,text
0,Owl City,Hey Anna,"Hey Anna, it's just a boy you won't understand..."
1,Coldplay,Rainy Day,Then there was rain \r\nThe sky wore a veil o...
2,Don Moen,"Two Hands, One Heart",[Verse 1] \r\nWhat can I give \r\nWhat can I...
3,Europe,Cherokee,"They lived in peace, not long ago \r\nA might..."
4,Red Hot Chili Peppers,Dani California,Getting born in the state of Mississippi \r\n...
5,Moody Blues,Meanwhile,Just in a simple conversation \r\nYou can hea...
6,Men At Work,Ain't Nothing Gonna Hold Me Down,"Last night I had the strangest dream, \r\nI s..."
7,Bruce Springsteen,Dream Baby Dream,Dream baby dream \r\nDream baby dream \r\nDr...
8,Eric Clapton,Got You On My Mind,"I've got you on my mind, I'm feeling kind of s..."
9,Billie Holiday,Always,I'll be loving you always \r\nWith a love tha...


In [11]:
df['text'][0]

"Hey Anna, it's just a boy you won't understand if you try  \r\nSo, hey Anna, smile when he's holding your hand tonight  \r\nHey Anna, remember when you're wishing away your life  \r\nHey, hey Anna, everything is going to turn out just fine  \r\n  \r\nDa-da-da, da-da-da, da-da-whoa-oh  \r\nDa-da-da, da-da-da, da-da-whoa-oh  \r\nDa-da-da, da-da-da, da-da-whoa-oh  \r\nDa-da-da, da-da-da, da-da-whoa-oh  \r\n  \r\nSuch a hot crush  \r\nMakes your face blush  \r\nBut try to take your time  \r\n  \r\n'Cause every goose chase  \r\nMakes your heart race  \r\nAnd you wonder why'  \r\n  \r\nSo wait for the home run  \r\nYou'll find the right one  \r\nAnd you'll figure this all out  \r\n  \r\nLife is a mind game  \r\nLove is a blind date  \r\nAnd it's making your heart pound  \r\n  \r\nHey Anna, it's just a boy you won't understand if you try  \r\nSo, hey Anna, smile when he's holding your hand tonight  \r\nHey Anna, remember when you're wishing away your life  \r\nHey, hey Anna, everything is go

In [12]:
# df = df.sample(5000)

In [13]:
df.shape

(5000, 3)

Text Cleaning/ Text Preprocessing

In [14]:
df['text'] = df['text'].str.lower().replace(r'^\w\s', ' ').replace(r'\n', ' ', regex = True)

In [15]:
import nltk
from nltk.stem.porter import PorterStemmer
stemmer = PorterStemmer()

def tokenization(txt):
    tokens = nltk.word_tokenize(txt)
    stemming = [stemmer.stem(w) for w in tokens]
    return " ".join(stemming)

ModuleNotFoundError: No module named 'nltk'

In [ ]:
df['text'] = df['text'].apply(lambda x: tokenization(x))

In [16]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [17]:
tfidvector = TfidfVectorizer(analyzer='word',stop_words='english')
matrix = tfidvector.fit_transform(df['text'])
similarity = cosine_similarity(matrix)

In [18]:
similarity[0]

array([1.        , 0.00472966, 0.00749653, ..., 0.01257709, 0.17123701,
       0.00654018])

In [26]:
df[df['song'] == 'Good Old Days']

,artist,song,text
1034,P!nk,Good Old Days,"1, 2, 3, 4, 5 years go by, \r i don't really ..."
4405,Ziggy Marley,Good Old Days,irie days come on play \r let the angels fly ...


In [27]:
def recommendation(song_df):
    idx = df[df['song'] == song_df].index[0]
    distances = sorted(list(enumerate(similarity[idx])),reverse=True,key=lambda x:x[1])
    
    songs = []
    for m_id in distances[1:21]:
        songs.append(df.iloc[m_id[0]].song)
        
    return songs

In [28]:
recommendation('Good Old Days')

['Good Old Days',
 'Old Days',
 'Days Of 49',
 'Forty Days',
 'Lonely Days',
 'Better Days',
 'Old Time Rock And Roll',
 'Good, Good, Good',
 'On My Way Home',
 'Old Flame, New Fire',
 'Same Old Song And Dance',
 'Good To Yourself',
 'The Goods Gone',
 'Let The Good Times Roll',
 '30 Days',
 'Comes Love',
 'Feel Good',
 'On Days Like These',
 'Those Good Old Dreams',
 'Old Shep']

In [ ]:
import pickle
pickle.dump(similarity,open('similarity.pkl','wb'))
pickle.dump(df,open('df.pkl','wb'))